In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.relpath('../'))
import dunn_index
from sklearn import metrics
import numpy as np
from sklearn import cluster
%matplotlib inline

In [2]:
analys_personal_data = pd.read_csv('../../../Data/For_analysis_and_modeling/2nd_quadrimester/analys_personal_data.csv',sep='|')

In [3]:
analys_personal_data.head()

,expediente,cod_plan,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,2,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,D,25,300.6305,S,0.235294,0.176471,1.990064,4.175,5.7,3.84
1,3,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Formación Profesional,6.836,H,31,667.4115,S,0.312500,0.562500,1.566486,5.000,5.0,2.50
2,4,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Formación Profesional,6.836,D,28,91.0020,N,0.166667,0.444444,1.345324,3.825,5.2,2.53
3,5,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Selectividad,6.730,D,25,91.0020,N,0.263158,0.315789,1.850120,3.670,5.4,2.50
4,6,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,H,25,91.0020,N,1.000000,0.000000,1.339003,7.000,7.0,0.00


In [4]:
interval_cols = []

In [5]:
analys_personal_data_model = analys_personal_data.copy()

In [6]:
analys_personal_data_model.drop(['expediente','cod_plan','abandona'],axis=1,inplace=True)

In [7]:
analys_personal_data_model.head()

,des_plan,anio_apertura_expediente,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,6.836,D,25,300.6305,S,0.235294,0.176471,1.990064,4.175,5.7,3.84
1,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Formación Profesional,6.836,H,31,667.4115,S,0.312500,0.562500,1.566486,5.000,5.0,2.50
2,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Formación Profesional,6.836,D,28,91.0020,N,0.166667,0.444444,1.345324,3.825,5.2,2.53
3,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,6.730,D,25,91.0020,N,0.263158,0.315789,1.850120,3.670,5.4,2.50
4,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,6.836,H,25,91.0020,N,1.000000,0.000000,1.339003,7.000,7.0,0.00


In [8]:

note_bcket_array = np.array([5, 6.5, 8, 9.5, 10, 11.5, 13, 14])
analys_personal_data_model['nota_admision_def'] = pd.cut(
    analys_personal_data_model['nota_admision_def'], note_bcket_array, include_lowest=True)
interval_cols.append('nota_admision_def')

year_bcket_array = np.array([18, 20, 25, 30, 35, 60])
analys_personal_data_model['edad_acceso'] = pd.cut(
    analys_personal_data_model['edad_acceso'], year_bcket_array, include_lowest=True)
interval_cols.append('edad_acceso')


distance_bcket_array = np.array([0, 45, 90, 135, 250, 500, 1892])
analys_personal_data_model['distance'] = pd.cut(
    analys_personal_data_model['distance'], distance_bcket_array, include_lowest=True)
interval_cols.append('distance')

note_bcket_array = np.array([0, 1.5, 3, 4.5, 6, 7.5, 9, 10])
analys_personal_data_model['cum_median'] = pd.cut(
    analys_personal_data_model['cum_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_median')
note_bcket_array = np.array([0, 4.5, 6, 7.5, 9, 10])
analys_personal_data_model['cum_pass_median'] = pd.cut(
    analys_personal_data_model['cum_pass_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_median')
note_bcket_array = np.array([0, 1.5, 3, 5])
analys_personal_data_model['cum_fail_median'] = pd.cut(
    analys_personal_data_model['cum_fail_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_fail_median')

ratio_bcket_array = np.array([0, 0.25, 0.5, 0.75, 1])
analys_personal_data_model['cum_pass_ratio'] = pd.cut(
    analys_personal_data_model['cum_pass_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_ratio')

analys_personal_data_model['cum_absent_ratio'] = pd.cut(
    analys_personal_data_model['cum_absent_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_absent_ratio')

std_bcket_array = np.array([0, 0.05,1.15,1.25,2.15,3.25])
analys_personal_data_model['std_deviation'] = pd.cut(
    analys_personal_data_model['std_deviation'], std_bcket_array, include_lowest=True)
interval_cols.append('std_deviation')


In [9]:
analys_personal_data_model['cum_pass_ratio'].value_counts()

(-0.001, 0.25]    976
(0.25, 0.5]       458
(0.5, 0.75]       234
(0.75, 1.0]       105
Name: cum_pass_ratio, dtype: int64

In [10]:
def le_dataset(dset, le_cols, cat_cols):
    from sklearn import preprocessing
    for col in cat_cols:
        le = preprocessing.LabelEncoder()
        le.fit(dset[col].cat.categories)
        le_cols.append(le)
        dset[col] = le.transform(dset[col])

In [11]:
def inverse_le_dataset(dset, le_cols, cat_cols):
    from sklearn import preprocessing
    i = 0
    for col in cat_cols:
        le = le_cols[i]
        from sklearn import preprocessing
        dset[col] = le.inverse_transform(dset[col])
        i +=1

In [12]:
def get_dunn_index(data, labels):
    from sklearn.metrics.pairwise import euclidean_distances
    distances = euclidean_distances(data)
    return dunn_index.dunn(labels,distances)


In [13]:
analys_personal_data_model.dtypes

des_plan                      object
anio_apertura_expediente      object
convocatoria_acceso           object
des_acceso                    object
nota_admision_def           category
sexo                          object
edad_acceso                 category
distance                    category
becario                       object
cum_pass_ratio              category
cum_absent_ratio            category
std_deviation               category
cum_median                  category
cum_pass_median             category
cum_fail_median             category
dtype: object

In [14]:
for col in analys_personal_data_model.columns:
    if 'object' in str(analys_personal_data_model.dtypes[col]):
        analys_personal_data_model[col] = analys_personal_data_model[col].astype('category')



In [15]:
le_cols = []
cat_cols_le = [ 'convocatoria_acceso','nota_admision_def','edad_acceso','cum_median','cum_pass_median','cum_fail_median','cum_pass_ratio','cum_absent_ratio','std_deviation','distance']

In [16]:
analys_personal_data['cum_pass_ratio'].max()

1.0

In [17]:
analys_personal_data_labels = analys_personal_data_model.copy()
le_dataset(analys_personal_data_labels,le_cols,cat_cols_le)
analys_personal_data_labels = pd.get_dummies(analys_personal_data_labels)
analys_personal_data_labels.head()

,convocatoria_acceso,nota_admision_def,edad_acceso,distance,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median,...,anio_apertura_expediente_2016-17,des_acceso_Formación Profesional,des_acceso_Mayores de 25/40/45 años,des_acceso_Selectividad,des_acceso_Traslado de Expediente (Estudios Españoles),des_acceso_Título Universitario,sexo_D,sexo_H,becario_N,becario_S
0,4,1,1,4,0,0,3,2,1,2,...,0,0,0,1,0,0,1,0,0,1
1,4,1,3,5,1,2,3,3,1,1,...,0,1,0,0,0,0,0,1,0,1
2,4,1,2,2,0,1,3,2,1,1,...,0,1,0,0,0,0,1,0,1,0
3,4,1,1,2,1,1,3,2,1,1,...,0,0,0,1,0,0,1,0,1,0
4,4,1,1,2,3,0,3,4,2,0,...,0,0,0,1,0,0,0,1,1,0


In [18]:
from sklearn.preprocessing import MinMaxScaler

norm = MinMaxScaler().fit(analys_personal_data_labels)
analys_personal_data_labels[analys_personal_data_labels.columns] = norm.transform(analys_personal_data_labels)

In [19]:
kmeans = cluster.KMeans(n_clusters=2,random_state=70)
kmeans.fit(analys_personal_data_labels)
analys_personal_data_clust = analys_personal_data_labels.copy()
analys_personal_data_clust['labels'] = kmeans.predict(analys_personal_data_labels)
analys_personal_data['labels'] = analys_personal_data_clust['labels'].astype('category')


/home/franjmelchor/.virtualenvs/education_drop_clustering/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [20]:
metrics.silhouette_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


0.19935306568809394

In [21]:
metrics.calinski_harabasz_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


468.9974953832511

In [22]:
metrics.davies_bouldin_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


1.924942183063768

In [23]:
get_dunn_index(analys_personal_data_clust,analys_personal_data_clust['labels'])

0.4586080289804953

In [24]:
import pandas as pd
def get_impurity(dropout_cl0: pd.core.series.Series, dropout_cl1:pd.core.series.Series):
    impurity = -1
    n = len (dropout_cl0) + len (dropout_cl1)
    if len(dropout_cl0[dropout_cl0 == 0]) > len(dropout_cl0[dropout_cl0 == 1]):
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 0]))/n
    else:
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 1]))/n
    if len(dropout_cl1[dropout_cl1 == 0]) > len(dropout_cl1[dropout_cl1 == 1]):
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 0]))/n)
    else:
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 1]))/n)
        
    return impurity

In [25]:
analys_personal_data_clust['abandona'] = analys_personal_data['abandona']

In [26]:
get_impurity(analys_personal_data_clust[analys_personal_data_clust['labels']==0]['abandona'], analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'])

0.45741680767061477

In [27]:
analys_personal_data_clust[analys_personal_data_clust['labels']==0]['abandona'].value_counts()

0    475
1    440
Name: abandona, dtype: int64

In [28]:
analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'].value_counts()

1    487
0    371
Name: abandona, dtype: int64

In [29]:
analys_personal_data_clust[analys_personal_data_clust['labels']==0]['abandona'].value_counts(normalize=True)

0    0.519126
1    0.480874
Name: abandona, dtype: float64

In [30]:
analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'].value_counts(normalize=True)

1    0.567599
0    0.432401
Name: abandona, dtype: float64